<a href="https://colab.research.google.com/github/ecflorui/genesys-lab/blob/main/convnext_base_GENESYS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
from copy import deepcopy
from tqdm import tqdm

# ----------- Config -----------
BATCH_SIZE = 32
NUM_EPOCHS = 50
PATIENCE = 7
IMG_SIZE = 224
LEARNING_RATE = 5e-4
NUM_CLASSES = 5  # change if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------- Dataset Setup -----------

class ImageArrayDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples  # list of (image_path, npy_label_path)
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, npy_path = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = int(np.argmax(np.load(npy_path)))
        return image, label

# Collect samples from v5 and v6 (1)
folders = [
    "/content/drive/MyDrive/v5",
    "/content/drive/MyDrive/v6 (1)"
]

all_samples = []
for folder_path in folders:
    for fname in os.listdir(folder_path):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            base = os.path.splitext(fname)[0]
            img_path = os.path.join(folder_path, fname)
            npy_path = os.path.join(folder_path, base + '.npy')
            if os.path.exists(npy_path):
                all_samples.append((img_path, npy_path))

random.shuffle(all_samples)
val_split = 0.2
split_idx = int(len(all_samples) * (1 - val_split))
train_samples = all_samples[:split_idx]
val_samples = all_samples[split_idx:]

# ----------- Transforms -----------

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = ImageArrayDataset(train_samples, transform=train_transform)
val_dataset = ImageArrayDataset(val_samples, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ----------- Label Smoothing Loss -----------

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes

    def forward(self, x, target):
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(logprobs)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * logprobs, dim=-1))

# ----------- Model Setup -----------

model = timm.create_model("convnext_base", pretrained=True, num_classes=NUM_CLASSES)
model.to(device)

criterion = LabelSmoothingLoss(NUM_CLASSES, smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

# ----------- Training Loop -----------

best_model = None
best_val_acc = 0.0
epochs_no_improve = 0

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss, running_correct = 0.0, 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    train_acc = running_correct / total
    train_loss = running_loss / total

    # Validation
    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels).item()
            val_loss += loss.item() * inputs.size(0)
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    val_loss = val_loss / val_total

    scheduler.step()

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f}, "
          f"Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = deepcopy(model.state_dict())
        epochs_no_improve = 0
        torch.save(best_model, "best_convnext_base.pth")
        print("✅ Saved best model.")
    else:
        epochs_no_improve += 1
        print(f"📉 No improvement. Patience: {epochs_no_improve}/{PATIENCE}")
        if epochs_no_improve >= PATIENCE:
            print(f"⏹ Early stopping at epoch {epoch+1}. Best Val Acc: {best_val_acc:.4f}")
            break


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

Epoch 1 Training: 100%|██████████| 15/15 [04:41<00:00, 18.79s/it]


Epoch 1: Train Loss = 1.7150, Train Acc = 0.2878, Val Loss = 1.3706, Val Acc = 0.4492
✅ Saved best model.


Epoch 2 Training: 100%|██████████| 15/15 [01:36<00:00,  6.44s/it]


...

Epoch 24: Train Loss = 0.8394, Train Acc = 0.7974, Val Loss = 1.3429, Val Acc = 0.5339
📉 No improvement. Patience: 5/7


Epoch 25 Training: 100%|██████████| 15/15 [01:38<00:00,  6.59s/it]


Epoch 25: Train Loss = 0.8894, Train Acc = 0.7783, Val Loss = 1.3316, Val Acc = 0.5932
📉 No improvement. Patience: 6/7


Epoch 26 Training: 100%|██████████| 15/15 [01:38<00:00,  6.56s/it]


Epoch 26: Train Loss = 0.8804, Train Acc = 0.7548, Val Loss = 1.3311, Val Acc = 0.5847
📉 No improvement. Patience: 7/7
⏹ Early stopping at epoch 26. Best Val Acc: 0.6525
